In [1]:
import pytest
import ipytest

ipytest.config(rewrite_asserts=True, magics=True)

current_notebook = 'pytest-101.ipynb'

__file__ = current_notebook

# Pytest 101

## Create test

Now you know how to do... it's so simple.

```python
def test_something():
    #blabla
    assert something
```

## Customize the assertion message

You could use the message feature of standard `assert`

```python 
assert condition, message
```

In [2]:
%%run_pytest[clean] -qq

def test_something():
    # given
    actual = 'A'
    expected = 'B'
    
    assert actual == expected, f'You mess the situation, {actual} is not {expected}'


F                                                                        [100%]
=================================== FAILURES ===================================
________________________________ test_something ________________________________

    def test_something():
        # given
        actual = 'A'
        expected = 'B'
    
>       assert actual == expected, f'You mess the situation, {actual} is not {expected}'
E       AssertionError: You mess the situation, A is not B
E       assert 'A' == 'B'
E         - A
E         + B

<ipython-input-2-94bb9a1d6309>:6: AssertionError


There's some more customization available **per argument type** see [official documentation](http://doc.pytest.org/en/latest/assert.html#making-use-of-context-sensitive-comparisons)

## Check a exception  is raised

_More difficult._

Image the _Processing Under Test_ must raise an exception

In [3]:
def check_planet_name(name: str):
    if name not in 'Mercury Venus Earth Mars Jupiter Saturn Uranus Neptune'.split():
        raise ValueError(f'{name} is not a planet')
    

In [4]:
%%run_pytest[clean] -qq

def test_is_a_planet():
    check_planet_name('Mercury')
    
def test_is_a_planet():
    check_planet_name('foo')

F                                                                        [100%]
=================================== FAILURES ===================================
_______________________________ test_is_a_planet _______________________________

    def test_is_a_planet():
>       check_planet_name('foo')

<ipython-input-4-3528dc69e803>:5: 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 

name = 'foo'

    def check_planet_name(name: str):
        if name not in 'Mercury Venus Earth Mars Jupiter Saturn Uranus Neptune'.split():
>           raise ValueError(f'{name} is not a planet')
E           ValueError: foo is not a planet

<ipython-input-3-903e962f3fdb>:3: ValueError


Pytest uses a context manager `pytest.raises`

In [5]:
%%run_pytest[clean] -qq

def test_is_a_planet():
    with pytest.raises(ValueError):
        check_planet_name('foo')

.                                                                        [100%]


In [6]:
%%run_pytest[clean] -qq
        
def test_is_a_planet_with_exception_detail():
    with pytest.raises(ValueError) as exception :
        check_planet_name('foo')        
    assert str(exception.value) == 'foo is not a planet'


.                                                                        [100%]


In [7]:
%%run_pytest[clean] -qq

def test_is_a_planet_with_matcher():
    with pytest.raises(ValueError, match=r'foo is*' ):
        check_planet_name('foo')

.                                                                        [100%]


For more information, please take a look at [pytest Raises Official Documentation](https://docs.pytest.org/en/latest/reference.html#pytest-raises)

## Fixtures

`Fixture` is a very powerful feature of pytest.

It make easy 

* factorisation and composition of common processing
* identification of setup and teardown steps
* to distinguish heavy and expensive operation for integration tests from lighweight and cheap operation for unit tests.


## What's a fixture ?

A factorized processing

Imagine a `Drone` class, powered by several `Engine`s and managed by a `DriverUnit`

In [8]:
class Drone:
    
    def __init__(self, engines, drive_unit):
        self.engines = engines
        self.drive_unit = drive_unit
        
        
    def start(self):
        for engine in self.engines:
            engine.start()
            
    def is_started(self):
        return all(engine.started for engine in self.engines)

To unit test the `Drone`  **in isolation of others object**, we create mock `Engine` and `DriveUnit`

In [9]:
class MockEngine:
    def start(self):
        self.status = 'started'
        
    def is_started(self):
        return self.status == 'started'
    
class MockDriveUnit:
    pass

In [10]:
%%run_pytest[clean] -q

def test_start_drone():
    # given
    engines = [MockEngine() for _ in range(1,4)]
    drone= Drone(engines=engines,drive_unit=MockDriveUnit())
    
    # When
    drone.start()
    
    #then
    assert all(engine.status == 'started' for engine in engines)

.                                                                        [100%]
1 passed in 0.02s


It's a little bit heavy to repeat no ?

What is the fundamental law of any programer ?

<center>DRY</center>
<div class="alert alert-info">
    <center>
    <b>
        <span style="font-size:larger;">DON'T<br>REPEAT<br>YOURSELF</span>
     </b>
    </center>
</div>

<center>Never,</center>
<center>ever</center>

So we can factorize the set up.

In [11]:
def init_drone():
    engines = [MockEngine() for _ in range(1,4)]
    drive_unit = MockDriveUnit()
    drone= Drone(engines=engines,drive_unit=drive_unit)
    return drone, engines, drive_unit

In [12]:
%%run_pytest[clean] -q

def test_start_drone():
    # given
    drone, engines, drive_unit = init_drone()
    
    # When
    drone.start()
    
    #then
    assert all(engine.status == 'started' for engine in engines)

.                                                                        [100%]
1 passed in 0.02s


_Pytest_ proposes a fixture mechanism to standardise this kind of factorisation.

**Firstly**

* Annotate a function with `@pytest.fixture` mark
* Make the function create the right context and return object of interest

**Secondly**

* Use the ***name*** of the function as argument of the test.
* Use the returned values from fixture

uhm ?

Illustration

In [13]:
import pytest

@pytest.fixture
def init_drone():
    engines = [MockEngine() for _ in range(1,4)]
    drive_unit = MockDriveUnit()
    drone= Drone(engines=engines,drive_unit=drive_unit)
    return drone, engines, drive_unit    

In [14]:
%%run_pytest[clean] -q

def test_start_drone(init_drone):
    # given
    drone, engines, drive_unit = init_drone
    
    # When
    drone.start()
    
    #then
    assert all(engine.status == 'started' for engine in engines)

.                                                                        [100%]
1 passed in 0.02s


_But it's the same, no ?_

Not really, because it's a declarative way of injecting dependency.  
For the sake of simplicity, we put the fixture near the test.

But the _fixture_ could be shared among several tests and defined in other part of the project.

Pytest will collect and inject them according to the name of the parameter.

As user, we don't have to care **how** to inject, _Pytest_ does it for us.

## Fixture composition

What appends if the fixture needs another common function ?

That's simple: it use the same mechanism :)

`def <function_name>(<dependency_name>)`

In [15]:
%%run_pytest[clean] -qq

@pytest.fixture
def a():
    v = ['A']
    return v

@pytest.fixture
def b(a):
    a.append('B')
    return a

def test_ab(b):
    assert b == ['A', 'B']
    

.                                                                        [100%]


The _consummer_ can ask for as many dependencies as it need.

In [16]:
%%run_pytest[clean] -qq

def test_ab(a,b): # Use 2 dependencies
    
    first_list = a
    second_list = b
    
    assert first_list == ['A','B']
    assert second_list == ['A', 'B']

.                                                                        [100%]
